# 型変換と演算子オーバーロード


## キーポイント

* クラスのインターフェイスはヘッダファイルに書き、メンバ関数の定義はCPPファイルに書く
* 一般に、クラスのヘッダファイルとCPPファイルの名前は「クラス名.h」「クラス名.cpp」とする
* `::`記号は「スコープ解決演算子」といって、スコープに含まれるメンバを指定するときに使う
* 四則演算や`<<`、`>>`などの演算子も関数オーバーロードができる
* メンバ関数と普通の関数のどちらの形式でも定義できる場合、基本的には普通の関数で定義する
* 普通の関数は、両辺が暗黙の型変換の対象となるため、メンバ関数より汎用性が高い
* 比較演算子を定義するときは、汎用の比較を行うメンバ関数を定義するとよい


----

## 1 型変換

----


### 1.1 文字列クラスのインターフェイスを考える

`int`や`double`のような組み込み型には、さまざまな演算が定義されています。<br>
また、論理的に近い型同士では、相互に変換が可能です。<br>
例えば、`int`と`double`の演算では、コンパイラは自動的に`int`を`double`に変換します。

クラスの設計者は、クラスの生成、コピー、代入、破棄の内容を自由に決められます。<br>
自動的な変換を可能にするには、コピーと代入を適切に定義します。

本テキストでは、クラスの型変換と、それに関連する関数を作成し、組み込み型のように扱えるクラスを作成します。<br>
このクラスの名前は`EasyStr`(イージー・エスティーアール)`とします。

`EasyStr`のお手本として、標準ライブラリの`string`クラスを参考にします。<br>
`string`クラスは、さまざまなパターンの型変換と関連する関数を持っているからです。<br>
とはいえ、`string`クラスのインターフェイスは`vector`より多いので、作成するのは機能を絞ったバージョンになります。

それでは、必要なインターフェイスを決めるために、`string`クラスの使い方を振り返ってみましょう。

```cpp
// string型の変数宣言
string s;
string t = "abcdef";
string u = t;

// サイズと添字を使ったループ
for (int i = 0; i < (int)t.size(); i++) {
  cin >> t[i];
}

// イテレータを使う
sort(t.begin(), t.end());

// 要素の追加と削除
s.push_back('a');
s.pop_back();

// 文字列の代入
s = t;
s = "abc";

// 標準入出力の読み書き
cin >> s;
cout << s;

// 文字列の加算
s += t;
t = s + t;
s += "def";
t = s + "ghi";
t = "jkl" + s;

// 文字列の比較
s == t;
s != "abc";
s < t;
"def" > t;
s <= t;
t >= s;
```

上記のプログラムでは、`string`の以下の機能を使用しています。

1. パラメータなしの宣言
2. 文字列を指定して初期化
3. サイズの取得
4. 添字によるデータの読み書き
5. イテレータの取得
6. 要素の追加と削除
7. `string`型の代入
8. 文字列の代入
9. 標準入力からの読み込み
10. 標準出力への書き込み
11. `string`の加算代入
12. `string`の加算
13. 文字列の加算代入

`string`は`vector`と同様の機能に加えて、文字列に関する機能を持つので、そのぶんインターフェイスが多くなります。


#### ファイルの準備

本テキストでは、実際に`EasyStr`クラスを作成してもらいます。<br>
`EasyStr`クラスはクラステンプレート**ではない**ので、ヘッダファイルにインターフェイスを書き、ソースファイルにメンバ関数の定義を書きます。
そのために、`EasyStr.h`と`EasyStr.cpp`という2つのファイルを用意します。

`Google Colab`では、コードセルの先頭に`%%writefile ファイル名`というマジックコマンドを書くことでファイルを作成できます。

この下に、マジックコマンドを書いた`EasyStr.h`と`EasyStr.cpp`という、2つのコードセルをご用意しました。

タイトルの`EasyStr.h`、`EasyStr.cpp`という文字をクリックすると、右側にアイコンリストが表示されます。アイコンリストから「タブのミラーセル」という説明が出るアイコンをクリックして、セルを複製してください(他のアイコンをクリックしないように注意)。


In [ ]:
# @title EasyStr.h
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyStr.h


In [ ]:
# @title EasyStr.cpp
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyStr.cpp


ウィンドウの右側に、`EasyStr.h`と`EasyStr.cpp`という2つのタブが表示されていたらOKです。以後、「`EasyStr.h`(または`EasyStr.cpp`)に次のプログラムを追加(あるいは削除、変更)してください」というテキストを見たら、右に表示されている`EasyStr.h`、`EasyStr.cpp`タブのコードを変更してください。

変更後に`▶`ボタンをクリックして、ファイルを保存すること。

また、`EasyStr`クラスでは、動的メモリ管理を`EasyVec`クラスに任せるつもりです。<br>
残念ながら、Google Colabには、別のノートのセルを参照する仕組みがありません。そこで、手動でコピーを作成します。<br>
前回の「C++言語\_第17回\_クラスの設計と実装」から、自分で作成した`EasyVec.h`の内容をコピーし、以下のセルに貼り付けてください。


In [ ]:
# @title EasyVec.h
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyVec.h
// この下に、前回テキストからコピーしたEasyVec.hを貼り付けてください。


`EasyVec.h`の完成版を貼り付けたら、`▶`ボタンをクリックしてファイルを保存してください。<br>
保存したら「タブのミラーセル」アイコンをクリックして、セルの複製を作成してください。

今回、`EasyVec.h`には手を付けませんが、サーバとの接続が切れた場合は、`▶`ボタンをクリックして保存し直す必要があります。


### 1.3 EasyStrクラスの定義

インターフェイスが決まったので、次は`EasyStr`クラスを具体的に定義していきます。<br>
`string`はテンプレートを使っていないので、`EasyStr`クラスも普通のクラスとして定義します。

また、メモリの管理に`EasyVec`クラスを使うので、`EasyVec.h`をインクルードします。

`EasyStr.h`の`%%writefile`行の下に、次のプログラムを追加してください。追加したら`▶`ボタンを押して保存すること。

```cpp
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

/*
* 文字列クラス
*/
class EasyStr {
public:

private:
  EasyVec<char> data;
};

#endif // EASYSTR_H
```

>追加のしかたが分からない場合は、以下の「追加例」の「コードの表示」をクリックして、参考にしてください


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.h
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

/*
* 文字列クラス
*/
class EasyStr {
public:

private:
  EasyVec<char>  data;
};

#endif // EASYSTR_H

### 1.4 型の定義

`EasyVec`クラスと同様に、`EasyStr`クラスにも以下の型の定義があると共通のプログラムを書きやすくなります。

* iterator:&emsp;&emsp;&emsp;基本となるイテレータ型
* const_iterator: 指している値を書き換えられないイテレータ型
* size_type:&emsp;&emsp; 配列の大きさなどを表現する型
* value_type:&emsp;&ensp;コンテナに格納するデータ型

今回は、メモリ管理用に`EasyVec`クラスを使っているので、`EasyVec`クラスの型の定義を利用できます。

例えば、`iteretor`型の定義は、次のように書けます。

`using iterator = EasyVec<char>::iterator;`

`EasyVec<char>`と`iterator`の間にある2つのコロン記号(`::`)は、「スコープ解決演算子」といいます。<br>
スコープ解決演算子の機能は、メンバ関数やメンバ変数の「所属(スコープ)」を明示することです。

「スコープ」は`{`と`}`で囲んだ範囲のことでした。実は、構造体やクラスの定義も「スコープ」です。<br>
すべてのメンバは、構造体やクラスのスコープに含まれます。

`クラス名::メンバ名`のように書くことで、クラスのメンバを指定できます。

>構造体やクラスのスコープは「名前付きスコープ」と呼ばれます。<br>
>スコープ解決演算子の左側には「名前付きスコープ」しか書けません(名前のないスコープは互いに区別できないため)。

それでは、必要な型を追加しましょう。`EasyStr.h`に、次のプログラムを追加してください。

**diff**

<pre><code><div> /*
 * 文字列クラス
 */
 class EasyStr {
 public:
<font color="#3a3">+  // 型の定義
+  using iterator = EasyVec&lt;char>::iterator;
+  using const_iterator = EasyVec&lt;char>::const_iterator;
+  using size_type =  EasyVec&lt;char>::size_type;
+  using value_type =  EasyVec&lt;char>::value_type;
</font>
 private:
   EasyVec&lt;char&gt; data;
 };
</div></code></pre>


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.h
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

/*
* 文字列クラス
*/
class EasyStr {
public:
  // 型の定義
  using iterator = EasyVec::iterator;
  using const_iterator = EasyVec::const_iterator;
  using size_type =  EasyVec::size_type;
  using value_type =  EasyVec::value_type;

private:
  EasyVec<char>  data;
};

#endif // EASYSTR_H

### 1.5 コンストラクタとデストラクタ


#### コンストラクタとデストラクタの宣言

`EasyStr`型には、2つのコンストラクタ、コピーコンストラクタ、そしてデストラクタを定義します。<br>
ただし、ヘッダファイルには追加するのは宣言だけです。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> class EasyStr {
 public:
   // 型の定義
   using iterator = EasyVec&lt;char>::iterator;
   using const_iterator = EasyVec&lt;char>::const_iterator;
   using size_type = EasyVec&lt;char>::size_type;
   using value_type = EasyVec&lt;char>::value_type;
<font color="#3a3">+
+  // コンストラクタ
+  EasyStr() = default;
+  EasyStr(const char* s);
+
+  // コピーコンストラクタ
+  EasyStr(const EasyStr& other);
+
+  // デストラクタ
+  ~EasyStr() = default;
</font>
 private:
   EasyVec&lt;char&gt; data;
 };
</div></code></pre>

`EasyStr`唯一のメンバ変数である`data`の型は`EasyVec`で、このクラスはデフォルトコンストラクタによって適切に初期化されます。<br>
そのため、`EasyStr`のデフォルトコンストラクタでは何もする必要はありません。<br>
このような場合は`default`を指定して、コンパイラに作ってもらうのが定石(じょうせき)です。

文字列を受け取るコンストラクタは、`const char*`型を使って文字列を受け取ります。<br>

デストラクタは、`default`指定をしてコンパイラに任せます。動的メモリ管理は`EasyVec`クラスが全てやってくれるからです。


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.h
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

/*
* 文字列クラス
*/
class EasyStr {
public:
  // 型の定義
  using iterator = EasyVec::iterator;
  using const_iterator = EasyVec::const_iterator;
  using size_type =  EasyVec::size_type;
  using value_type =  EasyVec::value_type;

  // コンストラクタ
  EasyStr() = default;
  EasyStr(const char* s);

  // コピーコンストラクタ
  EasyStr(const EasyStr& other);

  // デストラクタ
  ~EasyStr() = default;

private:
  EasyVec<char>  data;
};

#endif // EASYSTR_H

#### コンストラクタの定義

次に、コンストラクタの定義を作成します。`EasyStr.cpp`に次のプログラムを追加してください。

>**!!!注意!!!**<br>
>以下のコードは`EasyStr.cpp`に書くこと。EasyStr.hには書かないこと。

**diff**

<pre><code><div><font color="#3a3">+#include "EasyStr.h";
+#include &lt;string.h&gt;
+using namespace std;
+
+// 文字列を受け取るコンストラクタ
+EasyStr::EasyStr(const char* s) : data(strlen(s))
+{
+  for (int i = 0; i < (int)data.size(); i++) {
+    data[i] = s[i];
+  }
+}
+
+// コピーコンストラクタ
+EasyStr::EasyStr(const EasyStr& other) : data(other.data)
+{
+}</font>
</div></code></pre>

メンバ関数を定義する場合は、関数名の部分を次のように「クラス名 コロン記号2つ メンバ関数名」と書きます。

```cpp
EasyStr::EasyStr
```

`::`記号は、型を宣言するときにも使った「スコープ解決演算子」です。<br>
メンバ関数やメンバ変数の「所属(スコープ)」を明示する機能があります。<br>
上記の関数定義の場合、「`::`の前にある`EasyStr`がクラス名、後ろにある`EasyStr`がコンストラクタ名」です。

「文字列を受け取るコンストラクタ」は、受け取った文字列を`EasyVec`型の`data`メンバ変数にコピーします。<br>
コピー先の`data`には「文字列の長さ」のサイズが必要なので、標準ライブラリの`strlen`関数を使ってサイズを計算しています。

>`strlen`関数を使うには、`string.h`をインクルードする必要があります。

最後に、コピーコンストラクタは、`EasyVec`クラスのコピーコンストラクタを使って、データをコピーします。<br>
コピーに必要な処理は全部`EasyVec`クラスがやってくれます。


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.cpp
#include "EasyStr.h";
#include <string.h>
using namespace std;

// 文字列を受け取るコンストラクタ
EasyStr::EasyStr(const char* s) : data(strlen(s))
{
  for (int i = 0; i < (int)data.size(); i++) {
    data[i] = s[i];
  }
}

// コピーコンストラクタ
EasyStr::EasyStr(const EasyStr& other) : data(other.data)
{
}

#### 暗黙の型変換

`int a = 1.5;`と書いたとき、`double`型である`1.5`は自動的に`int`型の`1`に変換されます。<br>
このような、自動的に型が変換される機能を「暗黙の型変換(あんもくのかたへんかん)」といいます。

自作のクラスで暗黙の型変換を有効にするには、コンストラクタに`explicit`キーワードを付けないようにします。<br>
例えば、`EasyStr`クラスのコンストラクタには`explicit`を付けていないので、暗黙の型変換が有効になっています

`EasyStr`クラスで暗黙の型変換を有効にした理由は、お手本である`string`クラスが暗黙の型変換を有効にしているからです。<br>
文字列を扱うクラスでは、コンパイラが、文字列リテラルからクラスへと自動的に変換してくれると便利なことが多いのです。

```cpp
void func(const EasyStr& s); // こういう関数があるとして

// const char*からの暗黙の型変換が許可されている場合、以下の2行は同じ意味になる
func("abc");
func(EasyStr("abc")); // 暗黙の型変換が無効だと、この書き方しかできない
```


### 1.6 サイズと添え字演算子

`size`メンバ関数は、`EasyStr`が保持する文字列の長さを返します。<br>
添え字演算子は、指定された添え字の位置へ参照を返します。

これらの関数を宣言しましょう。`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>
   // コンストラクタ
   EasyStr() = default;
   EasyStr(const char* s);

   // デストラクタ
   ~EasyStr() = default;
<font color="#3a3">+
+  // 文字列の長さ
+  size_type size() const;
+
+  // 添え字
+  value_type& operator[](size_type i);
+  const value_type& operator[](size_type i) const;
</font>
 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>



続いて、`size`メンバ関数と添え字演算子の定義を作成します。`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> // 文字列を受け取るコンストラクタ
 EasyStr::EasyStr(const char* s) : data(strlen(s) + 1)
 {
   for (int i = 0; i < (int)data.size(); i++) {
     data[i] = s[i];
   }
 }

 // コピーコンストラクタ
 EasyStr::EasyStr(const EasyStr& other) : data(other.data)
 {
 }
<font color="#3a3">+
+// 文字列の長さ
+EasyStr::size_type EasyStr::size() const
+{
+  return data.size();
+}
+
+// 添え字
+EasyStr::value_type& EasyStr::operator[](size_type i)
+{
+  return data[i];
+}
+
+// 添え字
+const EasyStr::value_type& EasyStr::operator[](size_type i) const
+{
+  return data[i];
+}</font>
</div></code></pre>

`size`メンバ関数は、`EasyVec`の`size`メンバ関数を呼び出し、結果をそのまま返します。

添え字演算子は、`EasyVec`クラスの添え字演算子を呼び出すだけです。


### 1.7 イテレータ

次に、イテレータを返す`begin`、`end`メンバ関数を宣言します。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>
   // 文字列の長さ
   size_type size() const;

   // 添え字
   value_type& operator[](size_type i);
   const value_type& operator[](size_type i) const;
<font color="#3a3">+
+  // イテレータ
+  iterator begin();
+  const_iterator begin() const;
+  iterator end();
+  const_iterator end() const;</font>

 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>


続いて、イテレータを返すメンバ関数の定義を作成します。<br>
`EasyStr`の場合、これらのメンバ関数は、`EasyVec`クラスの同名のメンバ関数を呼び出すだけです。

`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> // 添え字
 EasyStr::value_type& EasyStr::operator[](size_type i)
 {
   return data[i];
 }

 // 添え字
 const EasyStr::value_type& EasyStr::operator[](size_type i) const
 {
   return data[i];
 }
<font color="#3a3">+
+// 先頭のイテレータ
+EasyStr::iterator EasyStr::begin()
+{
+  return data.begin();
+}
+
+// 先頭のイテレータ
+EasyStr::const_iterator EasyStr::begin() const
+{
+  return data.begin();
+}
+
+// 終端のイテレータ
+EasyStr::iterator EasyStr::end()
+{
+  return data.end();
+}
+
+// 終端のイテレータ
+EasyStr::const_iterator EasyStr::end() const
+{
+  return data.end();
+}</font>
</div></code></pre>


### 1.8 要素の追加と削除

末尾に要素を追加する`push_back`メンバ関数と、末尾の要素を削除する`pop_back`メンバ関数、それから全て消去する`clear`メンバ関数の3種類を宣言しましょう。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // イテレータ
   iterator begin();
   const_iterator begin() const;
   iterator end();
   const_iterator end() const;
<font color="#3a3">+
+  // 末尾に要素を追加
+  void push_back(value_type c);
+
+  // 末尾の要素を削除
+  void pop_back();
+
+  // すべての要素を削除
+  void clear();
</font>
 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>



`push_back`、`pop_back`、`clear`の3つのメンバ関数の定義も、`EasyVec`クラスの同名のメンバ関数を呼び出すだけで完成します。<br>
`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> // 終端のイテレータ
 EasyStr::iterator EasyStr::end()
 {
   return data.end();
 }

 // 終端のイテレータ
 EasyStr::const_iterator EasyStr::end() const
 {
   return data.end();
 }
<font color="#3a3">+
+// 末尾に要素を追加
+void EasyStr::push_back(value_type c)
+{
+  data.push_back(c);
+}
+
+// 末尾の要素を削除
+void EasyStr::pop_back()
+{
+  data.pop_back();
+}
+
+// すべての要素を削除
+void EasyStr::clear()
+{
+  data.clear();
+}</font>
</div></code></pre>


### 1.9 コピー代入演算子

`EasyStr`クラスのインターフェイスには、次の2パターンの代入があります。

```cpp
EasyStr a = "line";
EasyStr b = "box";

a = b;        // 1. EasyStrをEasyStrに代入
b = "sphere"; // 2. 文字列をEasyStrに代入
```

ですが、実際には「`EasyStr`を代入する演算子」を定義するだけで十分で、文字列用の代入演算子は不要です。<br>
なぜなら文字列は、暗黙の型変換によって、自動的に`EasyStr`型に変換されるからです。


#### コピー代入演算子の宣言

それでは、コピー代入演算子を宣言しましょう。`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // コンストラクタ
   EasyStr() = default;
   EasyStr(const char* s);

   // コピーコンストラクタ
   EasyStr(const EasyStr& other);

   // デストラクタ
   ~EasyStr() = default;
<font color="#3a3">+
+  // コピー代入演算子
+  EasyStr& operator=(const EasyStr& other);
</font>
   // 文字列の長さ
   size_type size() const;

   // 添え字
   value_type& operator[](size_type i);
   const value_type& operator[](size_type i) const;
</div></code></pre>


#### コピー代入演算子の定義

続いて、コピー代入演算子の定義を行います。<br>
コピー代入演算子は、`EasyVec`クラスのコピー代入演算子を使ってデータをコピーします。<br>
`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> // 文字列を受け取るコンストラクタ
 EasyStr::EasyStr(const char* s) : data(strlen(s))
 {
   for (int i = 0; i < (int)data.size(); i++) {
     data[i] = s[i];
   }
 }

 // コピーコンストラクタ
 EasyStr::EasyStr(const EasyStr& other) : data(other.data)
 {
 }
<font color="#3a3">+
+// コピー代入演算子
+EasyStr& EasyStr::operator=(const EasyStr& other)
+{
+  data = other.data;
+  return *this;
+}</font>

 // 文字列の長さ
 EasyStr::size_type EasyStr::size() const
 {
   return data.size();
 }
</div></code></pre>


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.cpp
#include "EasyStr.h";
#include <string.h>
using namespace std;

// 文字列を受け取るコンストラクタ
EasyStr::EasyStr(const char* s) : data(strlen(s))
{
  for (int i = 0; i < (int)data.size(); i++) {
    data[i] = s[i];
  }
}

// コピーコンストラクタ
EasyStr::EasyStr(const EasyStr& other) : data(other.data)
{
}

// コピー代入演算子
EasyStr& EasyStr::operator=(const EasyStr& other)
{
  data = other.data;
  return *this;
}

// 文字列の長さ
EasyStr::size_type EasyStr::size() const
{
  return data.size();
}

// 添え字
EasyStr::value_type& EasyStr::operator[](size_type i)
{
  return data[i];
}

// 添え字
const EasyStr::value_type& EasyStr::operator[](size_type i) const
{
  return data[i];
}

// 先頭のイテレータ
EasyStr::iterator EasyStr::begin()
{
  return data.begin();
}

// 先頭のイテレータ
EasyStr::const_iterator EasyStr::begin() const
{
  return data.begin();
}

// 終端のイテレータ
EasyStr::iterator EasyStr::end()
{
  return data.end();
}

// 終端のイテレータ
EasyStr::const_iterator EasyStr::end() const
{
  return data.end();
}
// 末尾に要素を追加
void EasyStr::push_back(value_type c)
{
  data.push_back(c);
}

// 末尾の要素を削除
void EasyStr::pop_back()
{
  data.pop_back();
}

// すべての要素を削除
void EasyStr::clear()
{
  data.clear();
}

### 1.10 標準入出力の読み書き


#### istreamとostream

標準入力である`cin`は`istream`(アイ・ストリーム)クラスの変数で、標準入力を受け付けるように特別に初期化されています。<br>
同様に、標準出力`cout`は`ostream`(オー・ストリーム)クラスの変数で、標準出力を受け付けるように特別に初期化されています。<br>

このため、標準入出力に対してデータを読み書きする関数を書く場合は、`istream`または`ostream`クラスを引数にします。

入力文字列を`string`クラスの変数で受け取る場合、

```cpp
cin >> s; // s は string クラスの変数
```

のように書くのでした。<br>
ここで登場する`>>`記号も演算子です。演算子は書きかたが特殊な関数に過ぎないので、関数オーバーロードができます。<br>
`EasyStr`用に`>>`演算子を関数オーバーロードすれば、`EasyStr`に対して標準入出力からデータを読み書きできるようになります。

演算子を関数オーバーロードすることを「演算子オーバーロード」といいます。


#### 二項演算子(にこう・えんざんし)

`>>`や四則演算のように、両辺に値を持つ演算子は「二項演算子(にこう・えんざんし)」と呼ばれます。<br>
上記の`>>`演算子の関数宣言は、次のようになります。

```cpp
istream& operator>>(istream& cin, const EasyStr& s);
```

このように、二項演算子の関数宣言では、第1引数が式の左辺の型、第2引数が右辺の型になります。<br>
また、もし演算子をメンバ関数にしたい場合は、第1引数の型のメンバにします。第2引数の型のメンバにはできません。<br>
これはC++のルールで決まっているので、変えられません。

このルールのため、上記の`>>`演算子はメンバ関数にはできません。<br>
左辺の型である`istream`は標準ライブラリに含まれるクラスなので、勝手にメンバ関数を追加できないからです。<br>
このような場合は、メンバ関数ではなく、普通の関数として定義します。


#### `>>`演算子の宣言

まず、`istream`クラスを使えるように、`iostream`ヘッダファイルをインクルードしましょう。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> #include "EasyVec.h";
<font color="#3a3">+#include &lt;iostream&gt;</font>

 /*
 * 文字列クラス
 */
 class EasyStr {
 public:
   // 型の定義
   using iterator = EasyVec&lt;char>::iterator;
</div></code></pre>

次に、関数の引数と戻り型を決めます。`cin >> s`の例で考えると、左辺の`cin`が第1引数、右辺の`s`が第2引数に相当します。<br>
`cin`は`istream`クラス、`s`は`EasyStr`クラスです。どちらも変更されうるため、`const`は付けられません。<br>
これらを考え合わせると、引数の型は`istream&`と`EasyStr&`となります。

戻り型については、`cin >> s >> t;`のような記述を可能にしたいです。<br>
そして、`cin >> s >> t;`は`cin >> s; cin >> t;`と同じ挙動になってほしいです。

そのためには、`cin >> s`の戻り値は、`cin`と同等の型でなくてはなりません。<br>
この条件を満たす戻り値の型は、`istream&`になります。

それでは、`>>`演算子の宣言を追加しましょう。<br>
追加する位置は、`istream`だけでなく`EasyStr`クラスの定義も参照できなくてはなりません。<br>
そのため、定義位置はクラス定義より下に書く必要があります。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 末尾の要素を削除
   void pop_back();

   // すべての要素を削除
   void clear();

 private:
   EasyVec&lt;char&gt; data;
 };
<font color="#3a3">+
+// ストリームから文字列を読み込む
+std::istream& operator>>(std::istream& is, EasyStr& str);</font>

 #endif // EASYSTR_H
</div></code></pre>


#### `>>`演算子の定義

`>>`演算子を`string`と同じように機能させるには、空白を判定して読み込みを停止しなくてはなりません。<br>
そのために、以下の手順が必要となります。

1. 読み込み先の`EasyStr`を空(から)にする
2. 文字列の前にある空白を読み飛ばす
3. データ終端、または空白が来るまで文字列を読み込む

この手順を全部手作業で書くのは大変です。幸い、標準ライブラリにはこういうときに便利な関数が数多く用意されています。<br>
今回は以下の5つの関数を使います。

>```cpp
>istream& ws(istream& is)
>```
>
>`ws`(ダブリューエス、ホワイト・スペースの頭文字)関数は、空白文字を読み捨てるストリーム操作クラス(マニピュレータ)。<br>
>`ws(cin);`または`cin >> ws;`のように書くことで、入力から空白文字を読み捨てることができる。

>```cpp
>int istream::get()
>```
>
>`istream::get`(アイストリーム・ゲット)関数は、ストリームから1文字読み込み、読み込んだ文字を返す。<br>
>終端文字列の場合は`char_traits<char>::eof()`を返す。

>```cpp
>tempalte<typename T>
>int char_traits<T>::eof()
>```
>
>`char_traits::eof`(チャートレイツ・イーオーエフ)関数は、終端文字を返す。

>```cpp
>int isspace(int c);
>```
>
>`isspace`(イズ・スペース)関数は、引数で渡された文字が「空白文字」なら`0`以外、空白文字でなければ`0`を返します。

>```cpp
>void istream::unget()
>```
>
>`istream::unget`(アイストリーム・アンゲット)関数は、最後に読み込んだ1文字をストリームに戻して、読まなかったことにします。

これらのうち`isspace`関数だけは`ctype.h`ヘッダファイルに宣言されています(他は`iostream`にある)。<br>
そこで、まずは`ctype.h`をインクルードしましょう。`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> #include "EasyStr.h";
 #include &lt;string.h&gt;
<font color="#3a3">+#include &lt;ctype.h&gt;</font>
 using namespace std;

 // 文字列を受け取るコンストラクタ
 EasyStr::EasyStr(const char* s) : data(strlen(s))
 {
   for (int i = 0; i < (int)data.size(); i++) {
     data[i] = s[i];
   }
 }
</div></code></pre>

それでは、`>>`演算子を定義しましょう。
`EasyStr.cpp`に、`>>`演算子の定義を追加してください。

**diff**

<pre><code><div> // 末尾の要素を削除
 void EasyStr::pop_back()
 {
   data.pop.back();
 }

 // すべての要素を削除
 void EasyStr::clear()
 {
   data.clear();
 }
<font color="#3a3">+
+// ストリームから文字列を読み込む
+istream& operator>>(istream& is, EasyStr& str) {
+  // 読み込み先のEasyStrを空にする  
+  str.clear();
+
+  // 文字列の前にある空白を読み飛ばす
+  is >> ws;
+
+  // データ終端、または空白が来るまで文字列を読み込む
+  while (is) {
+    int c = is.get();
+    if (c == char_traits&lt;char&gt;::eof()) {
+      break; // データ終端に到達したので読み込み終了
+    }
+    else if (isspace(c)) {
+      is.unget(); // ストリームに文字を戻す
+      break; // 空白を見つけたので読み込み終了
+    }
+    str.push_back((char)c); // 文字を追加
+  }
+
+  return is;
+}</font>
</div></code></pre>


#### `<<`演算子の宣言

次は、`EasyStr`を`cout`に出力できるようにします。これには`<<`演算子をオーバーロードします。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 末尾の要素を削除
   void pop_back();

   // すべての要素を削除
   void clear();

 private:
   EasyVec&lt;char&gt; data;
 };

 // ストリームから文字列を読み込む
 std::istream& operator>>(std::istream& is, EasyStr& str);
<font color="#3a3">+
+// ストリームに文字列を出力する
+std::ostream& operator<<(std::ostream& os, const EasyStr& str);</font>

 #endif // EASYSTR_H
</div></code></pre>

`<<`演算子も、`>>`演算子と同じ理由でメンバ関数にはできないため、普通の関数として宣言します。


#### `<<`演算子の定義

`EasyStr`クラスの文字列データは、実際には`EasyVec`型のメンバ変数`data`に格納されています。

`EasyStr.cpp`に、`<<`演算子の定義を追加してください。

**diff**

<pre><code><div>   // データ終端、または空白が来るまで文字列を読み込む
   while (is) {
     int c = is.get();
     if (c == char_traits&lt;char&gt;::eof()) {
       break; // データ終端に到達したので読み込み終了
     }
     else if (isspace(c)) {
       is.unget(); // ストリームに文字を戻す
       break; // 空白を見つけたので読み込み終了
     }
     str.push_back((char)c); // 文字を追加
   }

   return is;
 }
<font color="#3a3">+
+// ストリームに文字列を出力する
+ostream& operator<<(ostream& os, const EasyStr& str)
+{
+  for (auto i = str.begin(); i != str.end(); i++) {
+    os << *i;
+  }
+
+  return os;
+}</font>
</div></code></pre>


### 1.11 文字列の連結

お手本となる`string`クラスの変数は、`+=`演算子や`+`演算子を使って「文字列を連結」することができます。<br>
そこで、`EasyStr`クラスも、これらの演算子で連結できるようにしましょう。


#### `+=`演算子の宣言

引数や戻り型を考える前に、これらの演算子をメンバ関数にするか、通常の関数にするかを決める必要があります。

まず、`a += b`という式を考えてみましょう。<br>
この式では、`+=`の左辺にある変数`a`が変更され、右辺の`b`は変更されません。

このことから、`+=`演算子は、変更される左辺のクラスのメンバ関数にできそうです。<br>
右辺は変更されないため、引数の型は`const EasyStr&`とするのが適当でしょう。

それと、`string`型の`+=`演算子の場合、戻り型は`string&`で、左辺に対応する引数が返されます。<br>
極力お手本に合わせて作りたいので、戻り型は`EasyStr&`としましょう。

>右辺を変更してしまう`+=`演算子を定義することは可能です。が、利用者が混乱するだけなので普通はやりません。

上記の方針に従って、文字列を連結する`+=`演算子を追加しましょう。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 末尾の要素を削除
   void pop_back();

   // すべての要素を削除
   void clear();
<font color="#3a3">+
+  // 文字列の連結
+  EasyStr& operator+=(const EasyStr& other);</font>

 private:
   EasyVec&lt;char&gt; data;
};

 // ストリームから文字列を読み込む
 std::istream& operator>>(std::istream& is, EasyStr& str);
</div></code></pre>


#### `+=`演算子の定義

文字列の連結するには、`data`メンバ変数に`push_back`を繰り返して、1つずつ文字を追加します。<br>
`EasyStr.cpp`に、`+=`演算子の定義を追加してください。

**diff**

<pre><code><div> // 末尾の要素を削除
 void EasyStr::pop_back()
 {
   data.pop_back();
 }

 // すべての要素を削除
 void EasyStr::clear()
 {
   data.clear();
 }
<font color="#3a3">+
+// 文字列の連結
+EasyStr& EasyStr::operator+=(const EasyStr& other)
+{
+  for (auto i = other.begin(); i != other.end(); i++) {
+    data.push_back(*i);
+  }
+  return *this;
+}</font>

 // ストリームから文字列を読み込む
 istream& operator>>(istream& is, EasyStr& str) {
   // 読み込み先のEasyStrを空にする  
   str.clear();

   // 文字列の前にある空白を読み飛ばす
   is >> ws;
</div></code></pre>



#### `+`演算子の宣言

次に`a + b`という式を考えてみます。<br>
この式の結果は`a`と`b`を連結した新しいオブジェクトになります。`a`と`b`の内容は変更されません。

この式では両辺がどちらも変更されず、重要性も等しい(と思われる)ので、メンバ関数にする必然がありません。<br>
このように、メンバ関数と普通の関数のどちらの形式でも定義できる場合、基本的には普通の関数で定義します。

引数の型は、両辺共に変更されないことから、どちらも`const EasyStr&`となります。<br>
また，式の結果として新しいオブジェクトを返す必要があるため、戻り型は参照なしの`EasyStr`とする必要があります。

上記の方針に従って、文字列を連結する`+`演算子を追加しましょう。`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> private:
   EasyVec&lt;char&gt; data;
};

 // ストリームから文字列の読み込む
 std::istream& operator>>(std::istream& is, EasyStr& str);

 // ストリームに文字列を出力する
 std::ostream& operator<<(std::ostream& os, const EasyStr& str);
<font color="#3a3">+
+// 文字列の連結
+EasyStr operator+(const EasyStr& a, const EasyStr& b);</font>

 #endif // EASYSTR_H
</div></code></pre>

また、普通の関数では両辺が暗黙の型変換の対象となるため、メンバ関数より汎用性が高くなります。<br>
例えば、次のような式を書くことができます。

```cpp
EasyStr s = "sss";
cout << "aaa" + s << endl; // `EasyStr("aaa") + s`に変換される
cout << s + "bbb" << endl; // `s + EasyStr("bbb")`に変換される
```


#### `+`演算子の定義

それでは、`+`演算子の定義を追加しましょう。<br>
`EasyStr.cpp`に、次のプログラムを追加してください。

**diff**

<pre><code><div> // ストリームに文字列を出力する
 ostream& operator<<(ostream& os, const EasyStr& str)
 {
   for (auto i = str.begin(); i != str.end(); i++) {
     os << *i;
   }

   return os;
 }
<font color="#3a3">+
+// 文字列の連結
+EasyStr operator+(const EasyStr& a, const EasyStr& b)
+{
+  return EasyStr(a) += b;
+}</font>
</div></code></pre>

`+`演算子は、`+=`演算子を利用して作成しています。<br>
まずコピーコンストラクタで引数`a`のコピーを作ります。<br>
そして、`+=`演算子によって作成したコピーと引数`b`を連結します。

この例のように、四則演算は、対応する複合代入演算子を利用すれば簡単に定義できることが多いです。


### 1.12 文字列の比較

`string`クラスの変数は、互いに比較できます。それなら、`EasyStr`クラスも比較できるべきでしょう。

比較演算子には次の6種類があります。

| 記号 | 意味 |
|:----:|:----:|
| == | 等しい |
| != | 等しくない |
| <  | より小さい |
| >  | より大きい |
| <= | 以下 |
| >= | 以上 |

しかし、6つも演算子があると、個別に定義するのはかなりの手間です。<br>
そこで、次の機能を持つ関数を定義し、すべての演算子でこの関数を呼び出すように設計します。

* AとBが等しい場合、`0`を返す
* A&lt;Bの場合、負の数を返す
* A&gt;Bの場合、正の数を返す


#### 共通の比較関数を追加する


比較関数の名前は`compare`(コンペア、「比較する」という意味)とします。<br>
`compare`メンバ関数は、引数として「別の`EasyStr`」を受け取り、比較結果を`int`型で返します。

`EashStr.h`に、`EasyStr`同士の比較を行うメンバ関数宣言を追加してください。

**diff**

<pre><code><div>   // すべての要素を削除
   void clear();

   // 文字列の連結
   EasyStr& operator+=(const EasyStr& other);
<font color="#3a3">+
+  // 文字列の比較
+  int compare(const EasyStr& other) const;</font>

 private:
   EasyVec&lt;char&gt; data;
};

 // ストリームから文字列を読み込む
 std::istream& operator>>(std::istream& is, EasyStr& str);
</div></code></pre>

`compare`メンバ関数は、2つの文字列を比較するだけで、それらを変更することはありません。<br>
そこで、引数と関数の両方に`const`キーワードを付けることで、何も変更しないことを明示しています。

さて、お手本とする`string`クラスの比較関数は、文字列を辞書順で比較するために、戻り値を次の手順で決定します。

1. 先頭から文字を比較し、異なる文字を見つけたら2つの文字の差を戻り値とする
    * 例えば`"abc"`と`"acc"`を比較した場合、戻り値は`'b' - 'c'`、つまり`-1`になる
    * 辞書順で考えると「`"abc"`のほうが`"acc"`より小さい」ことをあらわす
2. どちらかの文字数が足りなくなり、そこまでの文字がすべて等しかった場合、文字数の差を返す。
    * 例えば`"abcd"`と`"abc"`を比較した場合、戻り値は`4 - 3`になる
    * 辞書順で考えると「文字数の少ないほうが小さい」ことをあらわす

`EasyStr`クラスでも、同じ方針で比較することにします。<br>
`EasyStr.cpp`に、`compare`メンバ関数の定義を追加してください。

**diff**

<pre><code><div> // 文字列の連結
 EasyStr& EasyStr::operator+=(const EasyStr& other)
 {
   data.pop_back(); // 終端文字を削除
   for (auto i = other.begin(); i != other.end(); i++) {
     data.push_back(*i);
   }
   data.push_back(0); // 終端文字を追加

   return *this;
 }
<font color="#3a3">+
+// 文字列の比較
+// 戻り値 0: *this と other は等しい
+//        0より小さい: *this は other より小さい
+//        0より大きい: *this は other より大きい
+int EasyStr::compare(const EasyStr& other) const
+{
+  const int s = size();
+  const int os = other.size();
+  for (int i = 0; i < s && i < os; i++) {
+    if (data[i] != other.data[i]) {
+      return data[i] - other.data[i];
+    }
+  }
+  return s - os;
+}</font>

 // ストリームから文字列を読み込む
 istream& operator>>(istream& is, EasyStr& str) {
   // 読み込み先の`EasyStr`を空にする  
   str.clear();

   // 文字列の前にある空白を読み飛ばす
   is >> ws;
</div></code></pre>




#### 比較演算子を追加する

比較演算子は「両辺とも変更しない二項演算子」です。そのため、普通の関数として宣言します。<br>
`EasyStr.h`に6つの比較演算子の宣言を追加してください。

**diff**

<pre><code><div> // ストリームから文字列を読み込む
 std::istream& operator>>(std::istream& is, EasyStr& str);

 // ストリームに文字列を出力する
 std::ostream& operator<<(std::ostream& os, const EasyStr& str);

 // 文字列の連結
 EasyStr operator+(const EasyStr& a, const EasyStr& b);
<font color="#3a3">+
+// 比較演算子
+bool operator==(const EasyStr& a, const EasyStr& b);
+bool operator!=(const EasyStr& a, const EasyStr& b);
+bool operator<(const EasyStr& a, const EasyStr& b);
+bool operator>(const EasyStr& a, const EasyStr& b);
+bool operator<=(const EasyStr& a, const EasyStr& b);
+bool operator>=(const EasyStr& a, const EasyStr& b);</font>

 #endif // EASYSTR_H
</div></code></pre>

続いて、`EasyStr.cpp`に比較演算子の定義を追加してください。

**diff**

<pre><code><div> // ストリームに文字列を出力する
 ostream& operator<<(ostream& os, const EasyStr& str)
 {
   for (auto i = str.begin(); i != str.end(); i++) {
     os << *i;
   }

   return os;
 }

 // 文字列の連結
 EasyStr operator+(const EasyStr& a, const EasyStr& b)
 {
   return EasyStr(a) += b;
 }

<font color="#3a3">+// 比較演算子
+bool operator==(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) == 0;
+}
+
+bool operator!=(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) != 0;
+}
+
+bool operator<(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) < 0;
+}
+
+bool operator>(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) > 0;
+}
+
+bool operator<=(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) <= 0;
+}
+
+bool operator>=(const EasyStr& a, const EasyStr& b)
+{
+  return a.compare(b) >= 0;
+}</font>
</div></code></pre>

これらの比較演算子の定義では、`compare`メンバ関数の戻り値を`0`と比較することで、適切な結果を得ています。

なお、`compare`と`0`の比較記号が、演算子と同じ記号なのは偶然ではありません。<br>
これは、お手本とした`string`クラスの比較が、比較演算子が定義しやすいように設計されているためです。

`compare`メンバ関数を`string`クラスと同じ方針で設計したので、同じように比較演算子が定義しやすくなっているわけです。


### EasyStrの実装例


In [ ]:
# @title 🔒️EasyStr.hの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyStr.h
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"
#include <iostream>

/*
* 文字列クラス
*/
class EasyStr {
public:
  // 型の定義
  using iterator = EasyVec<char>::iterator;
  using const_iterator = EasyVec<char>::const_iterator;
  using size_type = EasyVec<char>::size_type;
  using value_type = EasyVec<char>::value_type;

  // コンストラクタ
  EasyStr() = default;
  EasyStr(const char* s);

  // コピーコンストラクタ
  EasyStr(const EasyStr& other);

  // デストラクタ
  ~EasyStr() = default;

  // コピー代入演算子
  EasyStr& operator=(const EasyStr& other);

  // 文字列の長さ
  size_type size() const;

  // 添え字
  value_type& operator[](size_type i);
  const value_type& operator[](size_type i) const;

  // イテレータ
  iterator begin();
  const_iterator begin() const;
  iterator end();
  const_iterator end() const;

  // 末尾に要素を追加
  void push_back(value_type c);

  // 末尾の要素を削除
  void pop_back();

  // すべての要素を削除
  void clear();

  // 文字列の連結
  EasyStr& operator+=(const EasyStr& other);

  // 文字列の比較
  int compare(const EasyStr& other) const;

private:
  EasyVec<char> data;
};

// ストリームから文字列を読み込む
std::istream& operator>>(std::istream& is, EasyStr& str);

// ストリームに文字列を出力する
std::ostream& operator<<(std::ostream& os, const EasyStr& str);

// 文字列の連結
EasyStr operator+(const EasyStr& a, const EasyStr& b);

// 比較演算子
bool operator==(const EasyStr& a, const EasyStr& b);
bool operator!=(const EasyStr& a, const EasyStr& b);
bool operator<(const EasyStr& a, const EasyStr& b);
bool operator>(const EasyStr& a, const EasyStr& b);
bool operator<=(const EasyStr& a, const EasyStr& b);
bool operator>=(const EasyStr& a, const EasyStr& b);

#endif // EASYSTR_H

In [ ]:
# @title 🔒️EasyStr.cppの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyStr.cpp
#include "EasyStr.h"
#include <string.h>
#include <ctype.h>
using namespace std;

// 文字列を受け取るコンストラクタ
EasyStr::EasyStr(const char* s) : data(strlen(s))
{
  for (int i = 0; i < (int)data.size(); i++) {
    data[i] = s[i];
  }
}

// コピーコンストラクタ
EasyStr::EasyStr(const EasyStr& other) : data(other.data)
{
}

// コピー代入演算子
EasyStr& EasyStr::operator=(const EasyStr& other)
{
  data = other.data;
  return *this;
}

// 文字列の長さ
EasyStr::size_type EasyStr::size() const
{
  return data.size();
}

// 添え字
EasyStr::value_type& EasyStr::operator[](size_type i)
{
  return data[i];
}

// 添え字
const EasyStr::value_type& EasyStr::operator[](size_type i) const
{
  return data[i];
}

// 先頭のイテレータ
EasyStr::iterator EasyStr::begin()
{
  return data.begin();
}

// 先頭のイテレータ
EasyStr::const_iterator EasyStr::begin() const
{
  return data.begin();
}

// 終端のイテレータ
EasyStr::iterator EasyStr::end()
{
  return data.end();
}

// 終端のイテレータ
EasyStr::const_iterator EasyStr::end() const
{
  return data.end();
}

// 末尾に要素を追加
void EasyStr::push_back(value_type c)
{
  data.push_back(c);
}

// 末尾の要素を削除
void EasyStr::pop_back()
{
  data.pop_back();
}

// すべての要素を削除
void EasyStr::clear()
{
  data.clear();
}

// 文字列の比較
int EasyStr::compare(const EasyStr& other) const
{
  const int s = size();
  const int os = other.size();
  const int n = min(s, os);
  for (int i = 0; i < n; i++) {
    if (data[i] != other.data[i]) {
      return data[i] - other.data[i];
    }
  }
  return s - os;
}

// 文字列の連結
EasyStr& EasyStr::operator+=(const EasyStr& other)
{
  for (auto i = other.begin(); i != other.end(); i++) {
    data.push_back(*i);
  }

  return *this;
}

// ストリームから文字列を読み込む
istream& operator>>(istream& is, EasyStr& str) {
  // 読み込み先の`EasyStr`を空にする
  str.clear();

  // 文字列の前にある空白を読み飛ばす
  is >> ws;

  // データ終端、または空白が来るまで文字列を読み込む
  while (is) {
    int c = is.get();
    if (c == char_traits<char>::eof()) {
      break; // データ終端に到達したので読み込み終了
    }
    else if (isspace(c)) {
      is.unget(); // ストリームに文字を戻す
      break; // 空白を見つけたので読み込み終了
    }
    str.push_back((char)c); // 文字を追加
  }

  return is;
}

// ストリームに文字列を出力する
ostream& operator<<(ostream& os, const EasyStr& str)
{
  for (auto i = str.begin(); i != str.end(); i++) {
    os << *i;
  }
  return os;
}

// 文字列の連結
EasyStr operator+(const EasyStr& a, const EasyStr& b)
{
  return EasyStr(a) += b;
}

// 比較演算子
bool operator==(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) == 0;
}

bool operator!=(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) != 0;
}

bool operator<(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) < 0;
}

bool operator>(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) > 0;
}

bool operator<=(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) <= 0;
}

bool operator>=(const EasyStr& a, const EasyStr& b)
{
  return a.compare(b) >= 0;
}

In [ ]:
# @title 🔒️EasyVec.hの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyVec.h
#ifndef EASYVEC_H
#define EASYVEC_H
#include <memory>

/*
* 配列クラス
*/
template<typename T>
class EasyVec {
public:
  // 型の定義
  using iterator = T*;
  using const_iterator = const T*;
  using size_type = size_t;
  using value_type = T;

  // コンストラクタ
  EasyVec() = default;
  explicit EasyVec(size_t n) {
    // メモリを確保し、配列の先頭を設定
     first = alloc.allocate(n);

     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
  }

  // コピーコンストラクタ
  EasyVec(const EasyVec& other) {
    create(other);
  }

  // デストラクタ
  ~EasyVec() {
    destroy();
  }

  // コピー代入演算子
  EasyVec& operator=(const EasyVec& other) {
    // コピー元とコピー先が異なる場合だけコピーを実行
    if (this != &other) {

      if (size() != other.size()) {
        // サイズが異なる場合は配列を作り直す
        destroy();
        create(other);
      } else {
        // サイズが等しい場合はデータをコピーするだけ
        for (int i = 0; i < (int)size(); i++) {
          first[i] = other[i];
        }
      }
    } // if *this != other

    return *this; // このオブジェクトの参照を返す
  }

  // サイズの取得
  size_type size() const { return last - first; }

  // 添え字
  value_type& operator[](size_type i) { return first[i]; }
  const value_type& operator[](size_type i) const { return first[i]; }

  // イテレータ
  iterator begin() { return first; }
  const_iterator begin() const { return first; }
  iterator end() { return last; }
  const_iterator end() const { return last; }

  // 末尾に要素を追加する
  void push_back(const T& x) {
    // 未使用の部分が残っていなければ、配列のサイズを増やす
    if (last == cap) {
      grow();
    }

    // 配列の終端に要素を追加する
    std::construct_at(last, x);

    // 配列のサイズをひとつ増やす
    last++;
  }

  // 末尾の要素を削除する
  void pop_back() {
    // サイズが0でなければ要素をひとつ減らす
    if (size()) {
      std::destroy_at(last - 1);
      last--;
    }
  }

  // すべての要素を削除する
  void clear() {
    for (size_t i = 0; i < size(); i++) {
      std::destroy_at(first + i);
    }
    last = first;
  }

private:
  // 配列を作成する
  void create(const EasyVec& other) {
    // メモリを確保し、配列の先頭を設定
    const size_t n = other.size();
    first = alloc.allocate(n);

    // 要素をコピーコンストラクタで初期化
    for (size_t i = 0; i < n; i++) {
      std::construct_at(first + i, other[i]);
    }

    // 配列の終端を設定
    last = first + n;
    last = cap = first + n;
  }

  // 配列を破棄する
  void destroy() {
    // メモリを確保している場合のみ破棄と解放を実行
    if (first) {

      // 要素を破棄
      for (iterator i = first; i != last; i++) {
        std::destroy_at(i);
      }

      // メモリを解放
      alloc.deallocate(first, last - first);

      // ポインタを初期化
      first = last = nullptr;
      first = last = cap = nullptr;
    }
  }

  // 余分なメモリを確保する
  void grow() {
    // 確保するメモリのサイズを計算
    size_t n = 1; // メモリが確保されていない場合のサイズは1とする
    if (first) {
      // メモリが確保されている場合、元の2倍のメモリを確保する
      n = (cap - first) * 2;
    }

    // 新しいメモリを確保
    T* p = alloc.allocate(n);

    // 配列のすべてのデータを新しいメモリにコピー
    const size_t m = size();
    for (size_t i = 0; i < m; i++) {
      std::construct_at(p + i, first[i]);
    }

    // 元の配列を削除
    destroy();

    // 新しいメモリのアドレスとサイズをメンバ変数に反映
    first = p;
    last = p + m;
    cap = p + n;
  }

  T* first = nullptr; // 配列の先頭
  T* last = nullptr;  // 配列の終端
  T* cap = nullptr;   // 確保したメモリ領域の終端
  std::allocator<T> alloc; // メモリ管理用のオブジェクト
};

#endif // EASYVEC_H

----

## 2 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ EasyStrの初期化と添字

次の仕様に従って、本テキストで作成した`EasyStr`クラスを使ってN個の文字を読み込み、逆順で出力するプログラムを作成しなさい。

1. `int`型の変数`n`を宣言し、`cin`から`n`に文字数を読み込む
2. `EasyStr`型の変数`s`を宣言する
3. for文を使って、次の処理を`n`回繰り返す
    1. `EasyStr::value_type`型の変数`a`を宣言する
    2. `cin`から`a`に文字を読み込む
    3. `push_back`関数を使って、文字列`s`に変数`a`を追加する
4. for文を使って、次の処理を`n`回繰り返す
    1. `cout`に`s[n - 1 - i]`を出力する
5. `cout`に`endl`を出力する

>**【実行前の注意】**<br>
>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。<br>
>また、サーバとの接続が切れたときは、再接続後に上記の3ファイルを保存しなおす必要があります。

**入力例**

```txt
5
y d u t s
```

**出力例**

```txt
s t u d y
```


In [ ]:
%%writefile practice_01a.cpp
#include "EasyStr.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、1～5を実行するプログラムを書く
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp EasyStr.cpp && echo "この下をクリックして、文字数と文字のリストを入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "study\n1\ninternationalization") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp EasyStr.cpp && echo "5 y d u t s" | ./practice_01a && echo "1 1" | ./practice_01a && echo "20 n o i t a z i l a n o i t a n r e t n i" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include "EasyStr.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、1～5を実行するプログラムを書く
  int n;
  cin >> n;

  EasyStr s;
  for (int i = 0; i < n; i++) {
    EasyStr::value_type a;
    cin >> a;
    s.push_back(a);
  }

  for (int i = 0; i < n; i++) {
    cout << s[n - 1 - i];
  }
  cout << endl;
}

### ❓️問題２ EasyStrのイテレータ

N個の文字 $ A_1 $ ～ $ A_N $ が入力されます。<br>
本テキストで作成した`EasyStr`クラスを使って、配列にデータを読み込み、`sort`関数を使ってソートした結果を出力するプログラムを作成しなさい。最後に改行を出力すること。

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例**

```txt
8
c o m p u t e r
```

**出力例**

```txt
cemoprtu
```


In [ ]:
%%writefile practice_01b.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、N個の文字を読み込んで、ソートした結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp EasyStr.cpp && echo "この下をクリックして、を入力" && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "cemoprtu\nz\norst") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp EasyStr.cpp && echo "8 c o m p u t e r" | ./practice_01b && echo "1 z" | ./practice_01b && echo "4 s o r t" | ./practice_01b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include "EasyStr.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、N個の文字を読み込んで、ソートした結果を出力するプログラムを書く
  int n;
  cin >> n;

  EasyStr s;
  for (int i = 0; i < n; i++) {
    char a;
    cin >> a;
    s.push_back(a);
  }

  sort(s.begin(), s.end());

  for (auto i = s.begin(); i != s.end(); i++) {
    cout << *i;
  }
  cout << endl;
}

### ❓️問題３ EasyStrのコピー

本テキストで作成した`EasyStr`クラスについて、コピーコンストラクタおよびコピー代入演算子の動作を確認しましょう。

N個の文字 $ A_1 $ ～ $ A_N $ と、文字に足す数Mが入力されます。<br>
次の仕様にしたがって、プログラムを完成させなさい。

**プログラムの仕様**

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `int`型の変数`m`を宣言し、`cin`から`m`に「足す数」を読み込む
3. `EasyStr`型の変数`s`を宣言する
4. for文を使って、次の処理を`n`回行う
    1. `char`型の変数`a`を宣言し、`cin`から`a`に文字を読み込む
    2. `push_back`関数を使って、文字列`s`の末尾に`a`を追加する
5. `EasyStr`型の変数`t`を宣言し、変数`s`で初期化する
6. for文を使って、変数`t`の全ての要素に`m`を足す
7. 変数`s`に変数`t`を代入する
8. for文を使って、変数`s`の内容を全て`cout`に出力する
9. `cout`に改行を出力する

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例**

```txt
4 1
b n h m
```

**出力例**

```txt
coin
```


In [ ]:
%%writefile practice_01c.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、1～9を行うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp EasyStr.cpp && echo "この下をクリックして、文字数、足す数、文字のリストを入力" && ./practice_01c

In [ ]:
# @title 実行
!diff -Z <(echo -e "coin\ninternational\nzzz") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp EasyStr.cpp && echo "4 1 b n h m" | ./practice_01c && echo "13 -5 n s y j w s f y n t s f q" | ./practice_01c && echo "3 25 a a a" | ./practice_01c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01c.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、1～9を行うプログラムを書く
  int n, m;
  cin >> n >> m;

  EasyStr s;
  for (int i = 0; i < n; i++) {
    char a;
    cin >> a;
    s.push_back(a);
  }

  EasyStr t = s;
  for (int i = 0; i < n; i++) {
    t[i] += m;
  }

  s = t;

  for (int i = 0; i < n; i++) {
    cout << s[i];
  }
  cout << endl;
}

### ❓️問題４ EasyStrと標準入出力

本テキストで作成した`EasyStr`クラスの`>>`演算子と`<<`演算子を使って、2つの文字列を読み込み、逆順で出力するプログラムを作成しなさい。<br>
最後に改行を出力すること。

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例**

```txt
abc def
```

**出力例**

```txt
def abc
```


In [ ]:
%%writefile practice_01d.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、2つの文字列を逆順で出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp EasyStr.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "def abc\nrice curry") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp EasyStr.cpp && echo "abc def" | ./practice_02a && echo "curry rice" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、2つの文字列を逆順で出力するプログラムを書く
  EasyStr s, t;
  cin >> s >> t;

  cout << t << s << endl;
}

### ❓️問題５ EasyStrの連結

N個の文字列が入力されます。<br>
すべての文字列を入力の逆順で連結し、連結した文字列を出力するプログラムを作成しなさい。

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例**

```txt
3
One Shit Cat
```

**出力例**

```txt
CatShitOne
```


In [ ]:
%%writefile practice_02b.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、文字列を入力の逆順で連結して出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp EasyStr.cpp && echo "この下をクリックして、文字列の数と文字列のリストを入力" && ./practice_02b

In [ ]:
# @title 実行
!diff -Z <(echo -e "CatShitOne\nstandard\nPeopleWare") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp EasyStr.cpp && echo "3 One Shit Cat" | ./practice_02b && echo "8 d r a d n a t s" | ./practice_02b && echo "2 Ware People" | ./practice_02b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02b.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、文字列を入力の逆順で連結して出力するプログラムを書く
  int n;
  cin >> n;

  EasyStr a;
  for (int i = 0; i < n; i++) {
    EasyStr b;
    cin >> b;
    a = b + a;
  }

  cout << a << endl;
}

### ❓️問題６ EasyStrの比較

文字列A、文字列B、文字列Cの順で入力されます。<br>
文字列AとCを、文字列Bに対応する比較演算子を使って比較し、その結果が真なら文字列`Yes`、偽なら文字列`No`を出力するプログラムを作成しなさい。最後に改行を出力すること。

文字列Bに入力される値と、対応する演算は次のとおりです。

| 文字列B | 対応する演算 |
|:-------:|:----:|
| == | A == C |
| != | A != C |
| < | A < C |
| > | A > C |
| <= | A <= C |
| >= | A >= C |

なお、文字列Bに入力される記号は、上記の6種類のいずれかに限られます。他の記号が入力されることはないものとします。

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例（１）**

```txt
game != gamer
```

**出力例（１）**

```txt
Yes
```

**入力例（２）**

```txt
pen > sword
```

**出力例（２）**

```txt
No
```


In [ ]:
%%writefile practice_02c.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、文字列を比較した結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp EasyStr.cpp && echo "この下をクリックして、文字列、比較記号、文字列の3つを入力" && ./practice_02c

In [ ]:
# @title 実行
!diff -Z <(echo -e "Yes\nNo\nYes\nNo\nYes\nNo\nYes\nNo\nYes\nYes\nNo\nYes\nYes\nNo") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp EasyStr.cpp && echo "abstract == abstract" | ./practice_02c && echo "abc == aBc" | ./practice_02c && echo "game != gamer" | ./practice_02c && echo "abc != abc" | ./practice_02c && echo "sword > pen" | ./practice_02c && echo "pen > sword" | ./practice_02c && echo "abC < abc" | ./practice_02c && echo "abc < abC" | ./practice_02c && echo "abc >= abc" | ./practice_02c && echo "abc >= Abc" | ./practice_02c && echo "ab >= abc" | ./practice_02c && echo "abc <= abc" | ./practice_02c && echo "ab <= abc" | ./practice_02c && echo "abc <= ab" | ./practice_02c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02c.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、文字列を比較した結果を出力するプログラムを書く
  EasyStr a, b, c;
  cin >> a >> b >> c;

  bool d;
  if (b == "==") {
    d = a == c;
  } else if (b == "!=") {
    d = a != c;
  } else if (b == "<") {
    d = a < c;
  } else if (b == ">") {
    d = a > c;
  } else if (b == "<=") {
    d = a <= c;
  } else {
    d = a >= c;
  }

  if (d) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}

### ❓️問題７ 重複削除

N個の文字列が入力されます。<br>
重複する文字列を除いて、文字列を辞書順に出力するプログラムを作成しなさい。

>問題を解く前に、`EasyStr.h`, `EasyStr.cpp`, `EasyVec.h`の3つのセルを実行してファイルを保存すること。

**入力例（１）**

```txt
5
cc aaa bbb aaa ccc
```

**出力例（１）**

```txt
aaa bbb cc ccc
```

**入力例（２）**

```txt
26
E C W U F J S P M I X Z Q G Y K R D T V L N B A H O
```

**出力例（２）**

```txt
A B C D E F G H I J K L M N O P Q R S T U V W X Y Z
```


In [ ]:
%%writefile practice_03a.cpp
#include "EasyStr.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、文字列の重複をなくして辞書順に出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp EasyStr.cpp && echo "この下をクリックして、を入力" && ./practice_03a

In [ ]:
# @title 実行
!diff -Z <(echo -e "aaa bbb cc ccc\nA B C D E F G H I J K L M N O P Q R S T U V W X Y Z\ntest") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp EasyStr.cpp && echo "5 cc aaa bbb aaa ccc" | ./practice_03a && echo "26 E C W U F J S P M I X Z Q G Y K R D T V L N B A H O" | ./practice_03a && echo "3 test test test" | ./practice_03a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03a.cpp
#include "EasyStr.h"
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

int main() {
  // この下に、文字列の重複をなくして辞書順に出力するプログラムを書く
  int n;
  cin >> n;

  vector<EasyStr> v(n);
  for (int i = 0; i < n; i++) {
    cin >> v[i];
  }

  sort(v.begin(), v.end());

  auto itr = unique(v.begin(), v.end());
  v.erase(itr, v.end());

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}